<a href="https://colab.research.google.com/github/CamilleGreen5/Deep_Learning_with_Python/blob/master/Classification_dog_cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **LOAD DATA**

In [8]:
!pip install -q kaggle

In [43]:
from google.colab import files
import os
import glob
import shutil
import tensorflow as tf
import datetime
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
files.upload()

In [ ]:
# !rm -r sample_data
!mkdir kaggle
!cp kaggle.json kaggle
!rm kaggle.json
!chmod 600 kaggle/kaggle.json

In [ ]:
# !kaggle datasets list

In [ ]:
!kaggle competitions download -c 'dogs-vs-cats'

In [25]:
!unzip train.zip
!unzip test1.zip
!rm train.zip
!rm test1.zip

In [ ]:
!mkdir data
!mkdir data/train_data
!mkdir data/train_data/dog
!mkdir data/train_data/cat
!mkdir data/val_data
!mkdir data/val_data/dog
!mkdir data/val_data/cat
!mkdir data/test_data
!mkdir data/test_data/dog
!mkdir data/test_data/cat

train_dir = "data/train_data"
val_dir = "data/val_data"
test_dir = "data/test_data"

train_dog_dir = "data/train_data/dog"
train_cat_dir = "data/train_data/cat"
val_dog_dir = "data/val_data/dog"
val_cat_dir = "data/val_data/cat"
test_dog_dir = "data/test_data/dog"
test_cat_dir = "data/test_data/cat"

In [34]:
list_dog = glob.glob("./train/dog*")
list_cat = glob.glob("./train/cat*")
list_train_dog = list_dog[:2000]
list_train_cat = list_cat[:2000]
list_val_dog = list_dog[2000:3000]
list_val_cat = list_cat[2000:3000]
# print(len(list_train_dog), len(list_val_dog))
# print(len(list_train_cat), len(list_val_cat))
for path in list_train_dog:
    shutil.copy(path, train_dog_dir)
    os.remove(path)
for path in list_train_cat:
    shutil.copy(path, train_cat_dir)
    os.remove(path)
for path in list_val_cat:
    shutil.copy(path, val_cat_dir)
    os.remove(path)
for path in list_val_dog:
    shutil.copy(path, val_dog_dir)
    os.remove(path)

## **MODEL**

In [36]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard

In [50]:
def build_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])
    return model

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=20, class_mode='binary')
validation_generator = test_datagen.flow_from_directory(val_dir, target_size=(150, 150), batch_size=20, class_mode='binary')


In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model = build_model()
history = model.fit(train_generator, steps_per_epoch=100, epochs=30, validation_data=validation_generator, \
                    validation_steps=50, callbacks=[tensorboard_callback])
model.save("cat_and_dog_classifier.h5")

Epoch 1/30
  1/100 [..............................] - ETA: 0s - loss: 0.6924 - acc: 0.6000WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
100/100 [==============================] - 12s 116ms/step - loss: 0.6905 - acc: 0.5385 - val_loss: 0.6792 - val_acc: 0.5630
Epoch 2/30
100/100 [==============================] - 11s 113ms/step - loss: 0.6612 - acc: 0.5985 - val_loss: 0.6610 - val_acc: 0.5870
Epoch 3/30
100/100 [==============================] - 11s 113ms/step - loss: 0.6152 - acc: 0.6675 - val_loss: 0.5949 - val_acc: 0.6690
Epoch 4/30
100/100 [==============================] - 11s 111ms/step - loss: 0.5738 - acc: 0.7105 - val_loss: 0.6495 - val_acc: 0.6500
Epoch 5/30
 29/100 [=======>......................] - ETA: 5s - loss: 0.5607 - acc: 0.7224

In [ ]:
%tensorboard --logdir logs/fit --host localhost